# Model Training

History
- 4/8 
    - v1
- 4/9
    - modify data generator - flow_from_dataframe
    - implement multiple model training
- 4/10
    - cutmix
    - ensemble
- 4/12
    - detect flower and cut images
- 4/13
    - refactor to class
    - pseudo labeling
    
TODO
  - pseudo labeling

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, cv2, glob, datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from utils import *    
from trainner import *
import tensorflow as tf
import time
import joblib
from PIL import Image
os.environ['CUDA_VISIBLE_DEVICES'] = '1'



### 1. Prepare Data

In [ ]:

data_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_tr = pd.read_csv(os.path.join(data_floder,'label.csv'))
df_tr['category_str'] = df_tr['category'].astype('str')
df_tr['filepath']=df_tr['filename'].map(lambda x: os.path.join(data_floder,x))
df_tr['isAug']=False

#新增training_aug
aug_folder='/mnt/hdd1/Data/Competition/orchid/training_aug'
df_aug = pd.read_csv(os.path.join(aug_folder,'pseudo_label.csv'))
df_aug['category_str'] = df_aug['best_cls'].astype('str')
df_aug['filepath']=df_aug['filename'].map(lambda x: os.path.join(aug_folder,x))
df_aug['isAug']=True

#TODO: 新增test_public
#TODO: 新增test_private

df = pd.concat([df_tr,df_aug])

### 2. Training

In [ ]:
agent = Trainner()
#agent.pretrain_dir = '/mnt/hdd1/Model/TF_Pretrain'
agent.pretrain_dir = '/mnt/hdd1/Model/Competition/orchid/0412v1'
agent.model_dir = '/mnt/hdd1/Model/Competition/orchid'
agent.cutmix=True
agent.tiral_ver = '0413v1'
agent.cls_number=219
agent.lr=1e-6
agent.multi_gpu=False
EPOCH =30 

candidate_models = ['densenet169', 'densenet201', 'resnet101','xception','inception_resnet_v2','resnet152v2','resnet101v2']



In [ ]:
trtime_dict = {}
if agent.multi_gpu:
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        for model_name in candidate_models:
            df = shuffle(df)
            agent.training(model_name, df, EPOCH)
else:
    for model_name in candidate_models:
        df = shuffle(df)
        ts = time.time()
        agent.training(model_name, df, EPOCH)
        t = time.time()-ts
        trtime_dict[model_name]=t
        print(f'{model_name}: {t:.2f} sec')
    

trtimd_df = pd.DataFrame(trtime_dict)
joblib.dump(trtimd_df, f'model/trtime_{agent.tiral_ver}.pkl')


In [ ]:
#training time
display(trtimd_df)

#model loss/acc
model_dir = '/mnt/hdd1/Model/Competition/orchid/0412v1'
for p in glob.glob(os.path.join(model_dir,'*.jpg')):
    img=Image.open(p)
    display(img)